## Определение слов, являющихся фамилиями

In [1]:
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from nltk.stem.snowball import RussianStemmer

Читаем из файла обучающую выборку.

In [17]:
train_data = pd.read_csv('linear_train.txt', names = ["words", "target"])
train_data.head(10)

,words,target
0,Аалтонен,1
1,Аар,0
2,Аарон,0
3,ААРОН,0
4,Аарона,0
5,Аарона,1
6,Аароне,0
7,Ааронов,0
8,Аахена,0
9,Абабков,1


In [18]:
len(train_data)

101408

Используем библиотеку nltk.stem для стемминга (нахождения основы) русских слов, чтобы избавиться от приставок, окончаний и всего лишнего.

In [4]:
ps = RussianStemmer()
train_data_old = train_data.copy()
for i in range(len(train_data['words'])):
    train_data.set_value(i, 'words', ps.stem(train_data['words'][i].decode('utf-8')))

Очевидный недостаток стеммера -- это то, что он не сохраняет регистр, а в выборке присутствуют слова, для которых в зависимости от регистра разные ответы, например, "Грач" и "грач". Решим эту проблему просто: возьмем старый список слов (до стемминга) и заменим первую букву в каждом новом слове.

In [5]:
# Try to keep case after stemming
for i in range(len(train_data['words'])):
    train_data.set_value(i, 'words', 
                         train_data_old['words'][i].decode('utf-8')[0] + 
                         train_data['words'][i][1:])

In [6]:
train_data.head(10)

,words,target
0,Аалтон,1
1,Аар,0
2,Аарон,0
3,Аарон,0
4,Аарон,0
5,Аарон,1
6,Аарон,0
7,Аарон,0
8,Аах,0
9,Абабк,1


После стемминга в выборке образовалось много дубликатов, это видно даже по первым 10 словам. Удалим эти дубликаты.

In [7]:
# Drop duplicates
train_data.sort_values(['words', 'target'])
train_data.drop_duplicates(subset='words', inplace=True, keep='last')

In [8]:
train_data.head(10)

,words,target
0,Аалтон,1
1,Аар,0
7,Аарон,0
8,Аах,0
9,Абабк,1
12,абажур,0
13,Абажур,0
16,Абака,0
17,абак,0
18,Абакум,1


Читаем из файла тестовую выборку.

In [19]:
test_data = pd.read_csv('linear_test.txt', names = ["words"])
test_data.head(10)

,words
0,Аалто
1,ААР
2,Аара
3,Ааре
4,Аарон
5,Аароне
6,Ааронов
7,Аароном
8,Аароном
9,Аарону


In [20]:
len(test_data)

188920

Опять же применяем к ней стеммер и пытаемся сохранить регистр слова.

In [11]:
ps = RussianStemmer()
test_data_old = test_data.copy()
for i in range(len(test_data['words'])):
    test_data.set_value(i, 'words', ps.stem(test_data['words'][i].decode('utf-8')))

In [12]:
# Try to keep case after stemming
for i in range(len(test_data['words'])):
    test_data.set_value(i, 'words', 
                         test_data_old['words'][i].decode('utf-8')[0] + 
                         test_data['words'][i][1:])

In [13]:
test_data.head(10)

,words
0,Аалт
1,Аар
2,Аар
3,Аар
4,Аарон
5,Аарон
6,Аарон
7,Аарон
8,Аарон
9,Аарон


Обучаем нашу тестовую выборку, применив последовательно vectorizer, transformer и обучив модель LogisticRegression.

In [33]:
cv = CountVectorizer(min_df=1, ngram_range=(4, 11), analyzer='char_wb', binary=True, lowercase=False)
classifier = LogisticRegression()
pipeline = Pipeline([("vectorizer", cv), 
                     ("algo", classifier)
                    ])
pipeline.fit(train_data.words, train_data.target)

Pipeline(steps=[('vectorizer', CountVectorizer(analyzer='char_wb', binary=True, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(4, 11), preprocessor=None, stop_words=None,
 ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

Оценим качество двумя способами: ROC-AUC и accuracy.

Алгоритм прогонялся с разными наборами шагов преобразований данных, результат указан в комментариях.

In [34]:
np.mean(cross_val_score(pipeline, train_data.words, train_data.target, scoring='roc_auc'))

# 0.78095347831947004 - без всего
# 0.79682813243404016 - трансформер, без всего, lowercase=True
# 0.8092586099139063 - трансформер, без всего, lowercase=False

# Остальное со стеммером
# 0.73451051003839485 - трансформер, без keep case, без throw duplicates, lowercase=True
# 0.75900031347493313 - трансформер, с keep case, без throw duplicates, lowercase=False
# 0.77590498872723668 - трансформер, без keep case, с throw duplicates, lowercase=True
# 0.79372519723884449 - трансформер, с keep case, с throw duplicates, lowercase=False

0.80018925198676349

In [35]:
np.mean(cross_val_score(pipeline, train_data.words, train_data.target, scoring='accuracy'))

# 0.88803660074307755 - без всего
# 0.89394328765979747 - трансформер, без всего, lowercase=True
# 0.9018223278967108 - трансформер, без всего, lowercase=False

# Остальное со стеммером
# 0.89416026272770333 - трансформер, без keep case, без throw duplicates, lowercase=True
# 0.89738482917809836 - трансформер, с keep case, без throw duplicates, lowercase=False
# 0.84249198001769798 - трансформер, без keep case, с throw duplicates, lowercase=True
# 0.87934753020499612 - трансформер, с keep case, с throw duplicates, lowercase=False

0.89920916831982334

Никакие оптимизации, увы, не перебили самое простое решение.

Записываем результат в файл.

In [36]:
matrix = cv.transform(test_data.words)

In [37]:
output = pd.read_csv('linear_ans_example.txt', sep=',', names=['Id', 'Answer'], header=0)
output['Answer'] = 1 - classifier.predict_proba(matrix)
output.index.rename('Id', inplace=True)
output.to_csv('submission.csv', sep=',', index=False)

Посмотрим на то, что получилось. Выглядит неплохо :)

In [26]:
test_data = pd.read_csv('linear_test.txt', names = ["words"])
output['words'] = test_data['words']

print(output.loc[output['Answer'] > 0.99])

            Id    Answer         words
Id                                    
19          19  0.999362        Абаева
20          20  0.999988       Абаевым
41          41  0.999999     Абакумова
43          43  0.999866     Абакумову
44          44  1.000000    Абакумовым
45          45  0.992087       Абалкин
50          50  1.000000      Аббасова
51          51  0.999982      Аббасову
52          52  0.996252       Аббасом
103        103  1.000000  Абдрахманова
109        109  0.990432      Абдулина
112        112  0.998595    Абдуллаева
115        115  0.999966   Абдуллаевым
116        116  0.999356     Абдуллина
131        131  0.999839       Абелева
132        132  0.999999      Абелевым
179        179  0.999968       Абиевым
197        197  0.994546        Абнера
262        262  0.999926     Абрамкина
264        264  1.000000      Абрамова
270        270  0.998022   Абрамовичем
273        273  0.999999     Абрамовой
274        274  0.999968      Абрамову
275        275  1.000000 